In [1]:
import pandas as pd
import numpy as np
import math
from tqdm import tqdm

# External Data
## CS Rankings Author info

Taken from https://github.com/emeryberger/CSrankings/tree/gh-pages which is the repo behind https://csrankings.org 

Run `make generated-author-info.csv` which should make the csv. Each row contains the score for every author, year, and conference combination. This means that there are multiple rows for each professor.

## CS Professors

Taken from https://drafty.cs.brown.edu/csprofessors but they hide the data. So, after reading source code, exploit a [thing they left in](https://github.com/brownhci/drafty/blob/212bd995c857a34c74c7a71d67e1556c1ca7ea97/backend/src/controllers/datasharing.ts#L31) during development and use https://drafty.cs.brown.edu/data/csv/csprofessors/csprofessors_93318b344889ccef41d46b5f83d63de5 

## Placement Rank

Taken from https://drafty.cs.brown.edu/csopenrankings/placement-rank.html which is just copy and paste (and `M-x query-replace <tab> ,`). I think I could have done this myself, but why do that when someone else has already done the work. 

## Best Paper awards

A collection of best paper awards are listed on https://jeffhuang.com/best_paper_awards/

However, I needed to do some html parsing in order to get the data into a csv format. That is done in [another notebook](./best-paper.ipynb)

# Internal Data
## Preferences
This is a list of conferences and areas and my interest in them. This is used in order to create weights. It is manually generated

## Universities
This is a list of north american universities and their aliases (e.g. NYU and New York University). It is manually generated (hence why it is only north american universities)

## Statistic Weights
This is just a csv that contains weights for me to import and use in the final weighing stage.

# [Data Archive Download Link](http://store.sachiniyer.com/s/9NczHAmPJFiQgys/download/data.zip)


In [2]:
dfcsrankings = pd.read_csv("data/generated-author-info.csv")
dfcsrankings.describe()

,count,adjustedcount,year
count,199906.000000,199906.000000,199906.000000
mean,1.332511,0.384866,2012.689984
std,0.845756,0.281289,8.774593
min,1.000000,0.008696,1970.000000
25%,1.000000,0.200000,2008.000000
50%,1.000000,0.333330,2015.000000
75%,1.000000,0.500000,2020.000000
max,24.000000,4.977800,2023.000000


In [3]:
dfcsrankings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199906 entries, 0 to 199905
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   name           199906 non-null  object 
 1   dept           199906 non-null  object 
 2   area           199906 non-null  object 
 3   count          199906 non-null  float64
 4   adjustedcount  199906 non-null  float64
 5   year           199906 non-null  int64  
dtypes: float64(2), int64(1), object(3)
memory usage: 9.2+ MB


In [4]:
dfcsrankings.sample(3)

,name,dept,area,count,adjustedcount,year
151679,Sergey Levine,Univ. of California - Berkeley,siggraph-asia,1.0,0.200,2018
6965,Amaury Habrard,Université Jean Monnet,icml,1.0,0.250,2016
199114,Zhuozhao Li,SUSTech,hpdc,1.0,0.125,2020


In [5]:
dfcsprofs = pd.read_csv("data/csprofessors.csv")
dfcsprofs.describe()

,UniqueId,JoinYear
count,5630.000000,5512.000000
mean,3098.262877,2007.741473
std,1784.465144,11.730682
min,1.000000,1963.000000
25%,1569.250000,2001.000000
50%,3081.500000,2011.000000
75%,4623.750000,2017.000000
max,6212.000000,2024.000000


In [6]:
dfcsprofs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5630 entries, 0 to 5629
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   UniqueId    5630 non-null   int64  
 1   FullName    5629 non-null   object 
 2   University  5630 non-null   object 
 3   JoinYear    5512 non-null   float64
 4   SubField    5625 non-null   object 
 5   Bachelors   5331 non-null   object 
 6   Doctorate   5618 non-null   object 
dtypes: float64(1), int64(1), object(5)
memory usage: 308.0+ KB


In [7]:
dfcsprofs.sample(3)

,UniqueId,FullName,University,JoinYear,SubField,Bachelors,Doctorate
3204,3517,Sriram V. Pemmaraju,University of Iowa,1993.0,Algorithms & Complexity,Indian Institute of Technology Bombay,Virginia Tech
2025,2251,Ergun Akleman,Texas A&M University,1995.0,Visualization,Istanbul Technical University,Georgia Institute of Technology
3839,4206,Jia Rao,University of Texas at Arlington,2016.0,Operating Systems,Wuhan University,Wayne State University


In [8]:
dfplacement = pd.read_csv("data/placement-rank.csv", delimiter='|')
dfplacement.describe()

,num,size,bachelors,doctorate,placementscore
count,183.000000,115.000000,168.000000,159.000000,183.000000
mean,92.000000,45.121739,84.500000,80.000000,1.523333
std,52.971691,33.426050,48.641546,46.043458,1.664107
min,1.000000,1.000000,1.000000,1.000000,0.820000
25%,46.500000,22.500000,42.750000,40.500000,0.865000
50%,92.000000,39.000000,84.500000,80.000000,1.000000
75%,137.500000,60.000000,126.250000,119.500000,1.415000
max,183.000000,253.000000,168.000000,159.000000,15.400000


In [9]:
dfplacement.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183 entries, 0 to 182
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   num             183 non-null    int64  
 1   university      183 non-null    object 
 2   size            115 non-null    float64
 3   bachelors       168 non-null    float64
 4   doctorate       159 non-null    float64
 5   placementscore  183 non-null    float64
dtypes: float64(4), int64(1), object(1)
memory usage: 8.7+ KB


In [10]:
dfplacement.sample(3)

,num,university,size,bachelors,doctorate,placementscore
133,134,DePaul University,NaN,106.0,NaN,0.87
114,115,Missouri University of Science and Technology,NaN,97.0,141.0,0.90
149,150,Emory University,15.0,152.0,143.0,0.85


In [11]:
dfbestpaper = pd.read_csv("data/best-paper.csv", delimiter='|')
dfbestpaper = dfbestpaper.applymap(lambda x: x.strip() if isinstance(x, str) else x)
dfbestpaper.describe()

,Year,Coauthors
count,4815.000000,4815.000000
mean,2014.076843,5.455659
std,5.956105,3.199965
min,1996.000000,1.000000
25%,2010.000000,3.000000
50%,2015.000000,5.000000
75%,2019.000000,7.000000
max,2022.000000,26.000000


In [12]:
dfbestpaper.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4815 entries, 0 to 4814
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Author      4815 non-null   object
 1   University  4815 non-null   object
 2   Conference  4815 non-null   object
 3   Area        4815 non-null   object
 4   Year        4815 non-null   int64 
 5   Coauthors   4815 non-null   int64 
dtypes: int64(2), object(4)
memory usage: 225.8+ KB


In [13]:
dfbestpaper.sample(3)

,Author,University,Conference,Area,Year,Coauthors
1128,Gilad Katz,Ben-Gurion University of the Negev,CIKM,Knowledge Management,2019,5
989,Shie Mannor,Technion â Israel Institute of Technology,AAAI,Artificial Intelligence,2019,4
1922,Yuchi Tian & Baishakhi Ray,University of Virginia,FSE,Software Engineering,2017,1


In [14]:
dfpreference = pd.read_csv("data/preferences.csv")
dfpreference.describe()

,preference
count,99.000000
mean,2.747475
std,1.311981
min,1.000000
25%,2.000000
50%,3.000000
75%,4.000000
max,5.000000


In [15]:
dfpreference.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   conference  99 non-null     object
 1   preference  99 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.7+ KB


In [16]:
dfpreference.sample(3)

,conference,preference
31,hpca,3
48,siggraph,1
39,mobisys,2


In [17]:
university_alias = {}
with open('data/universities.csv', 'r') as file:
    for line in file:
        unis = line.split(',')
        unis.pop()
        first = unis.pop(0)
        university_alias[first] = unis 
count = 5
for k, v in university_alias.items():
    if count < 0:
        break
    count -= 1
    print(f"{k} {v}")
print(len(university_alias))

University of Kentucky []
Colorado State University []
University of Illinois at Chicago []
Duke University []
Florida International University []
UCCS []
186


In [18]:
def get_alias(uni):
    if uni in university_alias:
        return uni
    for k, v in university_alias.items():
        if uni in v:
            return k
    return None 

In [19]:
dfstatistics = pd.read_csv("data/statistics.csv")
dfstatistics.describe()

,weight
count,0.0
mean,NaN
std,NaN
min,NaN
25%,NaN
50%,NaN
75%,NaN
max,NaN


In [20]:
dfstatistics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   stat    84 non-null     object 
 1   weight  0 non-null      float64
dtypes: float64(1), object(1)
memory usage: 1.4+ KB


In [21]:
dfstatistics.sample()

,stat,weight
78,doctpaper_scoreclean_mean,NaN


# Methodology

I want to do a little better than following the rankings blindly, here are the factors I want to consider.

## Data

1. Professor Conference Score (essentially the csrankings.org score)
2. Best Paper Awards
3. Placement Rank (taken from the drafty placement rank, which is pagerank on professors original universities and then employment universities)
4. Variance of Professors

## Method

### Professor Score
A professor will be given a score based on their conference score and best papers. 

Both will have weights by:
- Will be weighted by logarithmically by the year published (the older, the less score)
- Will be weighted by how interested I am in the area (weights defined manually)

### Meta Scores

#### Placement Rank
A Placement Rank will be computed based on the professor score (the places where more important professors originate from matters more).

Logarithmic scale will be used to normalize the professor scores here.

#### Professor Score Variance
Variance of professor score will then be computed

#### Professor Score Sum
A sum of professors score will then be computed

### Final Score

I will take the Meta Scores, normalize them with min max scaling. Then I will apply some weights and get my results.



# Computing Professor Scores

- I will use the `generated-author-info` dataset for each professors to give their scores for each unique year-conference combination
- I will use the `best-paper` dataset for each professor to give a bonus for each professor with a best paper
- I will use `preference` dataset (which I manually created) to give weight preferences.


In [97]:
def year_weight(year):
    # 1970 - 2023
    year = 2023 - year + 3
    year *= 0.5
    return math.log(year)
#[year_weight(i) * 0.333 for i in range(1970, 2024)]

In [98]:
def area_weight(area):
    # score of 1-5
    area_score = dfpreference[dfpreference['conference'] == area]['preference'].values[0]
    return math.log(area_score + 1, 8)
#[math.log(i+1, math.e ** 0.125)*0.125 for i in range(1,6)]

In [99]:
names = []
def match_name(name, column, dataframe, one=False):
    clean_name = name.strip()
    last_four = clean_name[-4:]
    if last_four.isdigit():
        clean_name = clean_name[:-4]
    clean_name = clean_name.strip().lower()
    rows = dataframe[dataframe[column].str.strip().str.lower() == clean_name]
    if one:
        if rows.empty:
            return None
        return pd.DataFrame(rows.iloc[0])
    return rows

#match_name("Jiawei Han 0001", "FullName" , dfcsprofs)

In [100]:
#dfcsprofs[dfcsprofs['FullName'] == 'Jiawei Han']

In [26]:
dfauthorscore = pd.DataFrame(columns=['name', 'university', 'score'])

for index, row in tqdm(dfcsrankings.iterrows(), desc="Prof Scores", unit="item", total=len(dfcsrankings)):
    name = row['name']
    university = row['dept']

    year = row['year']
    area = row['area']

    yearweight = year_weight(year)
    areaweight = area_weight(area)


    count = row['adjustedcount'] * yearweight * areaweight

    existing_rows = dfauthorscore[dfauthorscore['name'] == name]
    if existing_rows.empty:
        dfauthorscore.loc[len(dfauthorscore)] = [name, university, count]
    else:
        for idx in existing_rows.index:
            dfauthorscore.at[idx, 'score'] += count

Prof Scores: 100%|██████████| 199906/199906 [05:50<00:00, 570.20item/s]


Prof Scores:   1%|██▎                                                                                                                                                                                                                                                                 | 1750/199906 [00:01<02:39, 1243.29item/s]

Prof Scores:   1%|██▍                                                                                                                                                                                                                                                                 | 1875/199906 [00:01<02:42, 1220.18item/s]

Prof Scores:   1%|██▌                                                                                                                                                                                                                                                                 | 2001/199906 [00:01<02:41, 1229.08item/s]

Prof Scores:   1%|██▊                                                                                                                                                                                                                                                                 | 2125/199906 [00:01<02:48, 1171.91item/s]

...

Prof Scores: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊| 199734/199906 [05:01<00:00, 471.25item/s]

Prof Scores: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊| 199782/199906 [05:01<00:00, 470.19item/s]

Prof Scores: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉| 199830/199906 [05:02<00:00, 461.56item/s]

Prof Scores: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉| 199877/199906 [05:02<00:00, 463.20item/s]

Prof Scores: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 199906/199906 [05:02<00:00, 661.56item/s]

In [27]:
dfpaperscore = pd.DataFrame(columns=['name', 'university', 'score'])

for index, row in tqdm(dfbestpaper.iterrows(), desc="Best Paper Scores", unit="item", total=len(dfbestpaper)):
    name = row['Author']
    university = row['University']

    year = row['Year']
    area = row['Area']
    coauthors = 1.0 / float(row['Coauthors'])

    yearweight = year_weight(year)
    areaweight = area_weight(area)

    count = 10 * yearweight * areaweight * coauthors

    existing_rows = dfpaperscore[dfpaperscore['name'] == name]
    if existing_rows.empty:
        dfpaperscore.loc[len(dfpaperscore)] = [name, university, count]
    else:
        for idx in existing_rows.index:
            dfpaperscore.at[idx, 'score'] += count

Best Paper Scores: 100%|██████████| 4815/4815 [00:10<00:00, 480.24item/s]


Best Paper Scores:  12%|█████████████████████████                                                                                                                                                                                         | 576/4815 [00:01<00:08, 516.75item/s]

Best Paper Scores:  13%|███████████████████████████▍                                                                                                                                                                                      | 628/4815 [00:01<00:08, 515.29item/s]

Best Paper Scores:  14%|█████████████████████████████▋                                                                                                                                                                                    | 682/4815 [00:01<00:07, 520.96item/s]

Best Paper Scores:  15%|████████████████████████████████▏                                                                                                                                                                                 | 737/4815 [00:01<00:07, 528.72item/s]

...

Best Paper Scores:  96%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌        | 4622/4815 [00:09<00:00, 482.98item/s]

Best Paper Scores:  97%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊      | 4672/4815 [00:09<00:00, 486.98item/s]

Best Paper Scores:  98%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉    | 4721/4815 [00:09<00:00, 480.99item/s]

Best Paper Scores:  99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████  | 4770/4815 [00:09<00:00, 475.49item/s]

Best Paper Scores: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4815/4815 [00:09<00:00, 507.54item/s]

In [112]:
dfauthorscore.describe()

,score
count,16202.000000
mean,5.289280
std,8.421237
min,0.007387
25%,0.831027
50%,2.286453
75%,6.095181
max,109.848482


In [111]:
dfauthorscore.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16202 entries, 0 to 16201
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   name        16202 non-null  object 
 1   university  16202 non-null  object 
 2   score       16202 non-null  float64
dtypes: float64(1), object(2)
memory usage: 1022.4+ KB


In [110]:
dfauthorscore.sample(3)

,name,university,score
491,Alexey Naumov,HSE University,0.189744
2100,Changick Kim,KAIST,1.941035
5274,Helen Petrie,University of York,2.024712


In [109]:
dfpaperscore.describe()

,score
count,4099.000000
mean,3.634394
std,4.174043
min,0.169683
25%,1.417166
50%,2.366560
75%,3.981688
max,50.220182


In [108]:
dfpaperscore.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4099 entries, 0 to 4098
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   name        4099 non-null   object 
 1   university  4099 non-null   object 
 2   score       4099 non-null   float64
dtypes: float64(1), object(2)
memory usage: 128.1+ KB


In [107]:
dfpaperscore.sample(3)

,name,university,score
2715,Hila Nachlieli,Hewlett Packard,2.688077
2700,Mark Segal,Google,1.648187
1275,Michael Schwarz,Graz University of Technology,0.695979


In [34]:
dfscore = dfauthorscore.copy()
dfscore.rename(columns={'score': 'author_score'}, inplace=True)
dfscore['paper_score'] = np.nan

for index, row in tqdm(dfpaperscore.iterrows(), desc="Merge Scores", unit="item", total=len(dfpaperscore)):
    name = row['name']
    university = row['university']
    count = row['score']

    existing_rows = match_name(name, 'name', dfscore)

    if existing_rows.empty:
        dfscore.loc[len(dfscore)] = [name, university, None, count]
    else:
        for idx in existing_rows.index:
            dfscore.at[idx, 'paper_score'] = count

Merge Scores: 100%|██████████| 4099/4099 [00:57<00:00, 71.69item/s]


Merge Scores:   8%|█████████████████▎                                                                                                                                                                                                     | 330/4099 [00:01<00:13, 282.14item/s]

Merge Scores:   9%|██████████████████▊                                                                                                                                                                                                    | 359/4099 [00:01<00:13, 279.86item/s]

Merge Scores:   9%|████████████████████▍                                                                                                                                                                                                  | 389/4099 [00:01<00:13, 282.57item/s]

Merge Scores:  10%|██████████████████████▏                                                                                                                                                                                                | 423/4099 [00:01<00:12, 296.41item/s]

...

Merge Scores:  97%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎     | 3990/4099 [00:14<00:00, 266.93item/s]

Merge Scores:  98%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊    | 4018/4099 [00:14<00:00, 269.59item/s]

Merge Scores:  99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏  | 4045/4099 [00:14<00:00, 266.36item/s]

Merge Scores:  99%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋ | 4073/4099 [00:14<00:00, 267.72item/s]

Merge Scores: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4099/4099 [00:14<00:00, 281.66item/s]

In [35]:
dfscore['university'] = dfscore['university'].apply(get_alias)
dfscore = dfscore.dropna(subset=['university'])
dfscore['name_lower'] = dfscore['name'].str.lower()
author_score_wnan = dfscore['author_score'].copy()
paper_score_wnan = dfscore['paper_score'].copy()
dfscore['combined_score'] = author_score_wnan.fillna(0) + paper_score_wnan.fillna(0)
dfscore.describe()

,author_score,paper_score,combined_score
count,6224.000000,1732.000000,7493.000000
mean,7.105932,4.113517,6.853321
std,10.088763,4.573070,9.962542
min,0.007792,0.169683,0.007792
25%,1.289504,1.527151,1.336447
50%,3.424615,2.682397,3.252147
75%,8.729862,4.622596,8.190972
max,109.848482,50.220182,116.461410


In [36]:
dfscore.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7493 entries, 4 to 19418
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name            7493 non-null   object 
 1   university      7493 non-null   object 
 2   author_score    6224 non-null   float64
 3   paper_score     1732 non-null   float64
 4   name_lower      7493 non-null   object 
 5   combined_score  7493 non-null   float64
dtypes: float64(3), object(3)
memory usage: 409.8+ KB


In [37]:
dfscore.sample(3)

,name,university,author_score,paper_score,name_lower,combined_score
14892,Xiao Wang 0012,Northwestern University,5.476842,NaN,xiao wang 0012,5.476842
14955,Xiaojun Shen,University of Missouri - Kansas City,1.121309,NaN,xiaojun shen,1.121309
6733,Joel A. Tropp,California Institute of Technology,3.183256,NaN,joel a. tropp,3.183256


# University dataframe

Now I will create a dataframe which has all people who currently work at the university, got their bachelors there, or got their doctorate there.

## Steps
1. Drop any universities that do not fit `get_alias`. This comprises of a lot of international universities that I don't have enough data on (and I don't want to apply to). Quite a lot of universities were filtered out in this stage.
2. Match the names and find out where profs got bachelors and phd from the `dfscore` and `dfcsprofs` dataframes. There will be a lot of misses here as well.
3. Modulate the dataframe to pivot off of universities with an array of authors instead of single author university pairings. 

## Name matching

Name matching will be done by lowering the case of everyones names and pivoting against the `name` field of the `dfscore` (which is from `dfcsrankings`) dataframe and the `FullName` field of the `dfcsprofs` dataframe. This will not end up with 100% matches (instead probably a lot less). There are also a lot of universities in both of these dataframes that wias filtered out with the `get_alias` function previously)


In [48]:
dfscore_placement = dfscore.copy()
dfscore_placement['placement'] = np.nan
dfscore_placement['bachelors'] = np.nan
dfscore_placement['doctorate'] = np.nan

dfcsprofs_placement = dfcsprofs.copy()
dfcsprofs_placement['placement'] = dfcsprofs_placement['University'].apply(get_alias)
dfcsprofs_placement['bachelors'] = dfcsprofs_placement['Bachelors'].apply(get_alias)
dfcsprofs_placement['doctorate'] = dfcsprofs_placement['Doctorate'].apply(get_alias)


for index, row in tqdm(dfscore_placement.iterrows(), desc="Merge Placement", unit="item", total=len(dfscore_placement)):
    prof_row = match_name(row['name'], 'FullName', dfcsprofs_placement, True)
    if prof_row is not None:
        dfscore_placement.loc[index, ['placement','bachelors','doctorate']] = dfcsprofs_placement.loc[prof_row.iloc[0].index[0], ['placement','bachelors','doctorate']]
        
dfscore_placement.describe()

Merge Placement: 100%|██████████| 7493/7493 [00:39<00:00, 190.19item/s]


,author_score,paper_score,combined_score
count,6224.000000,1732.000000,7493.000000
mean,7.105932,4.113517,6.853321
std,10.088763,4.573070,9.962542
min,0.007792,0.169683,0.007792
25%,1.289504,1.527151,1.336447
50%,3.424615,2.682397,3.252147
75%,8.729862,4.622596,8.190972
max,109.848482,50.220182,116.461410


In [49]:
dfscore_placement.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7493 entries, 4 to 19418
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name            7493 non-null   object 
 1   university      7493 non-null   object 
 2   author_score    6224 non-null   float64
 3   paper_score     1732 non-null   float64
 4   name_lower      7493 non-null   object 
 5   combined_score  7493 non-null   float64
 6   placement       3175 non-null   object 
 7   bachelors       1263 non-null   object 
 8   doctorate       2673 non-null   object 
dtypes: float64(3), object(6)
memory usage: 843.4+ KB


In [50]:
dfscore_placement.sample(3)

,name,university,author_score,paper_score,name_lower,combined_score,placement,bachelors,doctorate
14351,Vineet Bafna,Univ. of California - San Diego,7.667764,NaN,vineet bafna,7.667764,None,None,Pennsylvania State University
5084,Han-Wei Shen,Ohio State University,20.095160,NaN,han-wei shen,20.095160,Ohio State University,None,University of Utah
6909,Jonathan Ventura,UCCS,2.253934,NaN,jonathan ventura,2.253934,NaN,NaN,NaN


In [51]:
unique_unis = np.unique(np.concatenate([dfscore_placement['university'].dropna(),
                                        dfscore_placement['bachelors'].dropna(),
                                        dfscore_placement['doctorate'].dropna()]))

length = len(unique_unis)
inner_arrays = []
for _ in range(length):
    inner_arrays.append(np.array([]))

dfuniversity = pd.DataFrame({
    'university': unique_unis,
    'professors': inner_arrays,
    'bachelors': inner_arrays,
    'doctorate': inner_arrays
})
dfuniversity.sample(3)

,university,professors,bachelors,doctorate
124,University of Kentucky,[],[],[]
51,Montana State University,[],[],[]
104,University of Amsterdam,[],[],[]


In [52]:
for index, row in tqdm(dfscore_placement.iterrows(), desc="Merge Scores", unit="item", total=len(dfscore_placement)):
    name = row['name']
    university = row['university']
    bachelors = row['bachelors']
    doctorate = row['doctorate']

    existing_rows = dfuniversity[dfuniversity['university'] == university]
    if not existing_rows.empty:
        for idx in existing_rows.index:
            dfuniversity.at[idx, 'professors'] = np.append(dfuniversity.at[idx, 'professors'], name)

    existing_rows = dfuniversity[dfuniversity['university'] == bachelors]
    if not existing_rows.empty:
        for idx in existing_rows.index:
            dfuniversity.at[idx, 'bachelors'] = np.append(dfuniversity.at[idx, 'bachelors'], name)

    existing_rows = dfuniversity[dfuniversity['university'] == doctorate]
    if not existing_rows.empty:
        for idx in existing_rows.index:
            dfuniversity.at[idx, 'doctorate'] = np.append(dfuniversity.at[idx, 'doctorate'], name)


Merge Scores: 100%|██████████| 7493/7493 [00:08<00:00, 876.15item/s]


Merge Scores:  14%|████████████████████████████▉                                                                                                                                                                                        | 1020/7493 [00:00<00:05, 1133.67item/s]

Merge Scores:  15%|████████████████████████████████▏                                                                                                                                                                                    | 1134/7493 [00:01<00:05, 1111.54item/s]

Merge Scores:  17%|███████████████████████████████████▍                                                                                                                                                                                 | 1246/7493 [00:01<00:05, 1110.65item/s]

Merge Scores:  18%|██████████████████████████████████████▌                                                                                                                                                                              | 1358/7493 [00:01<00:05, 1110.29item/s]

...

Merge Scores:  94%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎             | 7010/7493 [00:06<00:00, 1207.90item/s]

Merge Scores:  95%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋          | 7131/7493 [00:06<00:00, 1206.49item/s]

Merge Scores:  97%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏      | 7253/7493 [00:06<00:00, 1210.14item/s]

Merge Scores:  98%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋   | 7377/7493 [00:06<00:00, 1218.05item/s]

Merge Scores: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7493/7493 [00:06<00:00, 1140.70item/s]

In [53]:
dfuniversity.sample(3)

,university,professors,bachelors,doctorate
28,Florida International University,"[A. Selcuk Uluagac, Amin Kharraz, Bogdan Carbu...",James F. O'Brien,[]
61,Northwestern University,"[Aggelos K. Katsaggelos, Aleksandar Kuzmanovic...","[Betty H. C. Cheng, Charles E. Hughes, Judith ...","[Abdol-Hossein Esfahanian, Carl K. Chang, Davi..."
32,Georgia Institute of Technology,"[Ada Gavrilovska, Ahmed Saeed 0001, Alan Ritte...","[Alex C. Snoeren, Anne Marie Piper, Annie I. A...","[Ada Gavrilovska, Adam O'Neill, Aggelos K. Kat..."


# Creating scores
While I am doing this, I will keep best paper and author score seperate. I will end up with 6 columns

- author score of people currently working there
- best paper score of people currently working there
- author score of people who got their bachelors there 
- best paper score of people who got their bachelors there 
- author score of people who got their doctorate there
- best paper score of people who got their doctorate there

I will start with empty arrays and then add to them as I lookup the scores of the authors from the `dfscore` dataframe. I also kept one array with the Nonetypes and one array without the Nonetypes to compute more statistics later.

In [54]:
dfuniversity['author_score'] = inner_arrays
dfuniversity['paper_score'] = inner_arrays
dfuniversity['bachauthor_score'] = inner_arrays
dfuniversity['bachpaper_score'] = inner_arrays
dfuniversity['doctauthor_score'] = inner_arrays
dfuniversity['doctpaper_score'] = inner_arrays
dfuniversity.sample(3)


,university,professors,bachelors,doctorate,author_score,paper_score,bachauthor_score,bachpaper_score,doctauthor_score,doctpaper_score
112,University of Colorado - Denver,"[Ilkyeun Ra, Min-Hyung Choi, Zhengxiong Li]",[],[],[],[],[],[],[],[]
126,University of Louisville,"[Adel Said Elmaghraby, Adrian P. Lauf, Antonio...",Anna Goldenberg,[],[],[],[],[],[],[]
2,Boise State University,"[Amit Jain, Bogdan Dit, Casey Kennington, Cath...",[],[],[],[],[],[],[],[]


In [55]:
def get_match(name, type):
    match = dfscore[dfscore['name_lower'] == name.lower()]
    if not match.empty:
        value = match.iloc[0][type]
        if not math.isnan(value):
            return value
    return None

def clean_none_value(value, clean):
    if value == None:
        if clean:
            return 0.0            
        else:
            return None
    return value


def lookup_scores(names, type, clean):
    values = []
    if isinstance(names, str):
        res = get_match(names, type)
        values.append(clean_none_value(res, clean))
        return values 
        
    if names.ndim == 0:
        res = get_match(names.item(), type)
        values.append(clean_none_value(res, clean))
        return values 
        
    for name in names:
        res = get_match(name, type)
        values.append(clean_none_value(res, clean))
    return values

def lookup_author_scores(names):
    return lookup_scores(names, "author_score", False)
def lookup_paper_scores(names):
    return lookup_scores(names, "paper_score", False)
def lookup_author_scores_clean(names):
    return lookup_scores(names, "author_score", True)
def lookup_paper_scores_clean(names):
    return lookup_scores(names, "paper_score", True)

In [56]:
tqdm.pandas(desc="Processing")
dfuniversity['author_score'] = dfuniversity['professors'].progress_apply(lookup_author_scores)
dfuniversity['paper_score'] = dfuniversity['professors'].progress_apply(lookup_paper_scores)
dfuniversity['bachauthor_score'] = dfuniversity['bachelors'].progress_apply(lookup_author_scores)
dfuniversity['bachpaper_score'] = dfuniversity['bachelors'].progress_apply(lookup_paper_scores)
dfuniversity['doctauthor_score'] = dfuniversity['doctorate'].progress_apply(lookup_author_scores)
dfuniversity['doctpaper_score'] = dfuniversity['doctorate'].progress_apply(lookup_paper_scores)
dfuniversity['author_scoreclean'] = dfuniversity['professors'].progress_apply(lookup_author_scores_clean)
dfuniversity['paper_scoreclean'] = dfuniversity['professors'].progress_apply(lookup_paper_scores_clean)
dfuniversity['bachauthor_scoreclean'] = dfuniversity['bachelors'].progress_apply(lookup_author_scores_clean)
dfuniversity['bachpaper_scoreclean'] = dfuniversity['bachelors'].progress_apply(lookup_paper_scores_clean)
dfuniversity['doctauthor_scoreclean'] = dfuniversity['doctorate'].progress_apply(lookup_author_scores_clean)
dfuniversity['doctpaper_scoreclean'] = dfuniversity['doctorate'].progress_apply(lookup_paper_scores_clean)

Processing: 100%|██████████| 185/185 [00:03<00:00, 59.15it/s]


In [57]:
dfuniversity.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 185 entries, 0 to 184
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   university             185 non-null    object
 1   professors             185 non-null    object
 2   bachelors              185 non-null    object
 3   doctorate              185 non-null    object
 4   author_score           185 non-null    object
 5   paper_score            185 non-null    object
 6   bachauthor_score       185 non-null    object
 7   bachpaper_score        185 non-null    object
 8   doctauthor_score       185 non-null    object
 9   doctpaper_score        185 non-null    object
 10  author_scoreclean      185 non-null    object
 11  paper_scoreclean       185 non-null    object
 12  bachauthor_scoreclean  185 non-null    object
 13  bachpaper_scoreclean   185 non-null    object
 14  doctauthor_scoreclean  185 non-null    object
 15  doctpaper_scoreclean   

# Computing Statistics

Here I am computing just a bunch of statistics. I iterate through each one of the arrays for all the different data type and compute statistics on each

It is not too complicated, but allows for me a lot of granularity of control in the weighing stage and computing the final score.

In [58]:
data_columns = [
    'author_score',
    'paper_score',
    'bachauthor_score',
    'bachpaper_score',
    'doctauthor_score',
    'doctpaper_score',
    'author_scoreclean', 
    'paper_scoreclean',
    'bachauthor_scoreclean',
    'bachpaper_scoreclean',
    'doctauthor_scoreclean',
    'doctpaper_scoreclean',
]

stats_columns = [
    'sum',
    'mean',
    'median',
    'min',
    'max',
    'var',
    'std'
]

In [59]:
add_columns = []
for d in data_columns:
    for s in stats_columns:
        dfuniversity[d + '_' + s] = np.nan
dfuniversity.sample(3)

,university,professors,bachelors,doctorate,author_score,paper_score,bachauthor_score,bachpaper_score,doctauthor_score,doctpaper_score,...,doctauthor_scoreclean_max,doctauthor_scoreclean_var,doctauthor_scoreclean_std,doctpaper_scoreclean_sum,doctpaper_scoreclean_mean,doctpaper_scoreclean_median,doctpaper_scoreclean_min,doctpaper_scoreclean_max,doctpaper_scoreclean_var,doctpaper_scoreclean_std
72,Rochester Institute of Technology,"[Adrienne Decker, Alexander Ororbia, Andrew Me...","[Andrew Pavlo, Vladimir Kolesnikov]",Joanna C. S. Santos,"[3.9132765275213393, 2.7421620428944644, 2.303...","[None, None, None, None, None, None, None, Non...","[7.9415732231362375, 5.914080725801838]","[1.0462974177791522, None]",[0.1670350624660491],[None],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,Northern Arizona University,"[Abolfazl Razi, Alex Groce, David Scott, Igor ...",Tajana Rosing,[],"[0.3087754025511043, 5.535036898626242, 0.8983...","[None, 2.5806856432051184, None, 0.80682580543...",[7.110562249670161],[None],[],[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
176,Virginia Commonwealth University,"[Alberto Cano 0001, Bartosz Krawczyk, Bridget ...",[],[],"[0.539724215975441, 0.8943177482126916, 0.2206...","[None, None, None, None, None, None, None, Non...",[],[],[],[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
dfuniversity.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 185 entries, 0 to 184
Data columns (total 100 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   university                    185 non-null    object 
 1   professors                    185 non-null    object 
 2   bachelors                     185 non-null    object 
 3   doctorate                     185 non-null    object 
 4   author_score                  185 non-null    object 
 5   paper_score                   185 non-null    object 
 6   bachauthor_score              185 non-null    object 
 7   bachpaper_score               185 non-null    object 
 8   doctauthor_score              185 non-null    object 
 9   doctpaper_score               185 non-null    object 
 10  author_scoreclean             185 non-null    object 
 11  paper_scoreclean              185 non-null    object 
 12  bachauthor_scoreclean         185 non-null    object 
 13  bach

In [90]:
def filter_array(data):
    data = list(filter(lambda x: x is not None, data))
    data = [x for x in data if not math.isnan(x)]
    data = [x for x in data if not np.isnan(x)]
    return data

def variance_hander(data):
    if len(data) <= 1:
        return 0
    return np.nanvar(data, ddof=1)

def get_handler(func, data):
    filtered_data = filter_array(data)
    if len(filtered_data) == 0:
        return 0.0 
    return func(filtered_data)

def get_sum(data):
    return get_handler(np.nansum, data)
def get_mean(data):
    return get_handler(np.nanmean, data)
def get_median(data):
    return get_handler(np.nanmedian, data)
def get_min(data):
    return get_handler(np.nanmin, data)
def get_max(data):
    return get_handler(np.nanmax, data)
def get_var(data):
    return get_handler(variance_hander, data)
def get_std(data):
    return get_handler(np.nanstd, data)

In [92]:
stat_function_map = {
    'sum': get_sum,
    'mean': get_mean,
    'median': get_median,
    'min': get_min,
    'max': get_max,
    'var': get_var,
    'std': get_std
}

In [94]:
count = 0 
for data in tqdm(data_columns, desc="Data Types", unit="item", total=len(data_columns)):
    for stat in stats_columns:
        stat_name = data + '_' + stat
        for index, row in dfuniversity.iterrows():
            values = row[data]
            dfuniversity.at[index, stat_name] = stat_function_map[stat](values) 
            
        sum_column = dfuniversity[stat_name].sum(skipna=True)
        dfuniversity[stat_name] = dfuniversity[stat_name].apply(lambda x: x / sum_column if pd.notna(x) else x)


Data Types: 100%|██████████| 12/12 [00:03<00:00,  3.70item/s]


# Final Rank
Here I take some weights I made manually and assign importance to each of the stats on a 1-5 scale. Then I can compute a final rank and see the colleges.

Also, just for fun I can see who my top profs are as well.

In [95]:
dfuniversity['final_score'] = np.nan
for index, row_uni in tqdm(dfuniversity.iterrows(), desc="Final Score", unit="item", total=len(dfuniversity)):
    score = 0
    for _, row_stat in dfstatistics.iterrows(): 
        statistic = row_stat['stat']
        weight = row_stat['weight']
        if not np.isnan(weight):
            partial_score = row_uni[statistic] * float(weight) * 0.2
            score += partial_score
    dfuniversity.at[index, 'final_score'] = score

Final Score: 100%|██████████| 185/185 [00:01<00:00, 133.56item/s]


In [96]:
# This is with no weights, I am still playing around with the final weights so this will be based on author_score_sum. Better weights are coming. 
dfuniversity.sort_values('author_score_sum', ascending=False).head(20)

,university,professors,bachelors,doctorate,author_score,paper_score,bachauthor_score,bachpaper_score,doctauthor_score,doctpaper_score,...,doctauthor_scoreclean_var,doctauthor_scoreclean_std,doctpaper_scoreclean_sum,doctpaper_scoreclean_mean,doctpaper_scoreclean_median,doctpaper_scoreclean_min,doctpaper_scoreclean_max,doctpaper_scoreclean_var,doctpaper_scoreclean_std,final_score
11,Carnegie Mellon University,"[Aarti Singh, Abhinav Gupta 0001, Alan M. Frie...","[Adam Chlipala, Adam Wierman, Alan L. Cox, Amy...","[Aaron C. Courville, Aarti Gupta, Abhinav Shri...","[7.495669736789303, 35.48159089255236, 37.0925...","[None, None, None, None, None, None, None, Non...","[17.794084473461275, 10.221338636427244, 10.88...","[2.199056938930018, 3.898716609892077, None, N...","[16.363269303720102, 16.209851394003906, 10.63...","[None, None, None, 3.898716609892077, 1.013124...",...,0.022631,0.020865,0.114232,0.014175,0.0,0.0,0.041679,0.022264,0.025634,0.0
45,Massachusetts Institute of Technology,"[Adam Belay, Adam Chlipala, Alan S. Willsky, A...","[Al Davis, Aleksandra Korolova, Ali José Masht...","[Aaron Sidford, Abe Davis, Abhi Shelat, Adam D...","[2.0743199057136525, 17.794084473461275, 10.01...","[2.3112981393783825, 2.199056938930018, None, ...","[4.839140830537145, 2.2228809626730164, 2.0926...","[None, None, None, None, None, None, None, Non...","[15.02025624529837, 2.437684577358209, 14.5295...","[3.8596925318888036, None, None, None, None, N...",...,0.024782,0.021843,0.155861,0.015283,0.0,0.0,0.043803,0.032973,0.031210,0.0
89,Univ. of California - Berkeley,"[Aditya G. Parameswaran, Alberto L. Sangiovann...",[],[],"[13.110848793332744, 42.830826721045824, 2.590...","[None, None, None, None, None, 2.3418223647650...",[],[],[],[],...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0
98,Univ. of Illinois at Urbana-Champaign,"[Adam Bates 0001, Aishwarya Ganesan, Alex Kirl...",[],[],"[5.486671024175045, 1.4155402466284959, 0.7823...","[None, None, None, None, None, None, None, Non...",[],[],[],[],...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0
76,Stanford University,"[Aaron Sidford, Alexander Aiken, Anshul Kundaj...","[Abe Davis, Alan J. Hu, Alvin Cheung, Andrew C...","[Aaron Stump, Adam Belay, Aditya G. Parameswar...","[15.02025624529837, 35.54670642244019, 1.69454...","[3.8596925318888036, None, None, 16.5635991664...","[2.437684577358209, 10.769116231332792, 8.0876...","[None, None, 2.3418223647650245, 11.1551280529...","[1.8074169876576829, 2.0743199057136525, 13.11...","[None, 2.3112981393783825, None, 1.97381431804...",...,0.041049,0.028090,0.105323,0.015290,0.0,0.0,0.037724,0.025116,0.027217,0.0
95,Univ. of California - San Diego,"[Aaron Schulman, Adalbert Gerald Soosai Raj, A...",[],[],"[2.966065301184756, 0.7197043343016801, 1.4027...","[None, None, None, 3.198707556995225, None, No...",[],[],[],[],...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0
21,Cornell University,"[Abe Davis, Adrian Sampson, Alexander M. Rush,...","[Aaron Stump, Adam J. Lee, Alexander Rakhlin, ...","[Alex Pothen, Alexander Travis Adams, Alin Dob...","[2.437684577358209, 2.8706979928756002, 16.848...","[None, None, None, None, 11.155128052942676, N...","[1.8074169876576829, 1.7101623586419767, 6.325...","[None, None, None, None, None, None, None, 3.8...","[2.7231833158830367, 1.2717136159194502, 9.771...","[None, None, 2.9735108991436507, None, None, N...",...,0.031855,0.024688,0.057503,0.016224,0.0,0.0,0.035162,0.032222,0.030757,0.0
32,Georgia Institute of Technology,"[Ada Gavrilovska, Ahmed Saeed 0001, Alan Ritte...","[Alex C. Snoeren, Anne Marie Piper, Annie I. A...","[Ada Gavrilovska, Adam O'Neill, Aggelos K. Kat...","[2.860489017477167, 0.7314841729510686, 8.1401...","[None, None, None, None, 3.0021797770820395, N...","[15.572263671591063, 5.245267189577871, 2.0877...","[3.198707556995225, 7.50127729841806, None, No...","[2.860489017477167, 6.104079274139918, 1.04306

In [67]:
dfscore.sort_values('combined_score', ascending=False).head(20)

,name,university,author_score,paper_score,name_lower,combined_score
2819,Dan Roth,University of Pennsylvania,94.035226,22.426184,dan roth,116.461410
10960,Philip S. Yu,University of Illinois at Chicago,109.848482,NaN,philip s. yu,109.848482
6494,Jiawei Han 0001,Univ. of Illinois at Urbana-Champaign,100.007356,NaN,jiawei han 0001,100.007356
8919,Martial Hebert,Carnegie Mellon University,92.681259,4.055012,martial hebert,96.736271
14087,Tuomas Sandholm,Carnegie Mellon University,96.699988,NaN,tuomas sandholm,96.699988
9860,Moshe Y. Vardi,Rice University,94.277997,NaN,moshe y. vardi,94.277997
11447,Ran Raz,Princeton University,43.606677,50.220182,ran raz,93.826859
2520,Christos Faloutsos,Carnegie Mellon University,77.823569,15.923711,christos faloutsos,93.747280
14297,Vijay Kumar 0001,University of Pennsylvania,92.118075,NaN,vijay kumar 0001,92.118075
3982,Eugene Charniak,Brown University,46.156553,44.804959,eugene charniak,90.961513
